# Bussiness understanding

# Data Understanding

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import re
import numpy as np

In [2]:
df = pd.read_csv('train.csv')
df.head()

,name,tweet
0,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,scotthamilton,is upset that he can't update his Facebook by ...
2,mattycus,@Kenichan I dived many times for the ball. Man...
3,ElleCTF,my whole body feels itchy and like its on fire
4,Karoli,"@nationwideclass no, it's not behaving at all...."


In [3]:
df.shape

(1600000, 2)

# Data Preparation

## Features use

* phone number
* mail address
* bank card number
* account number
* date of birth / age

## Preprocessing the data

### HTML decoding

In [4]:
df.tweet[279]

"Whinging. My client&amp;boss don't understand English well. Rewrote some text unreadable. It's written by v. good writer&amp;reviewed correctly. "

In [5]:
from bs4 import BeautifulSoup
example1 = BeautifulSoup(df.tweet[279], 'lxml')
print (example1.get_text())

Whinging. My client&boss don't understand English well. Rewrote some text unreadable. It's written by v. good writer&reviewed correctly. 


### Remove @mention

In [6]:
df.tweet[343]

'@TheLeagueSF Not Fun &amp; Furious? The new mantra for the Bay 2 Breakers? It was getting 2 rambunctious;the city overreacted &amp; clamped down '

In [7]:
re.sub(r'@[A-Za-z0-9]+','',df.tweet[343])

' Not Fun &amp; Furious? The new mantra for the Bay 2 Breakers? It was getting 2 rambunctious;the city overreacted &amp; clamped down '

### Remove non-ASCII character

In [8]:
df.tweet[226]

'Tuesdayï¿½ll start with reflection ï¿½n then a lecture in Stress reducing techniques. That sure might become very useful for us accompaniers '

In [9]:
''.join([i if ord(i) < 128 else ' ' for i in df.tweet[226]])

'Tuesday   ll start with reflection    n then a lecture in Stress reducing techniques. That sure might become very useful for us accompaniers '

### Remove Hashtag

In [10]:
df['tweet'][175]

"@machineplay I'm so sorry you're having to go through this. Again.  #therapyfail"

In [12]:
re.sub(r'(\s)#\w+', r'\1', df['tweet'][175])

"@machineplay I'm so sorry you're having to go through this. Again.  "

## Process on all tweet

In [13]:
from nltk.tokenize import WordPunctTokenizer
from bs4 import BeautifulSoup

tok = WordPunctTokenizer()

In [154]:
#at_rate = r'@[a-z0-9_]+'
at_rate = r'(?<=^|(?<=[^a-zA-Z0-9-_\.]))@([A-Za-z]+[A-Za-z0-9-_]+)'
url_link = r'https?://[A-Za-z0-9./]+'
hashtag = r'(\s)#\w+', r'\1' 
combine = r'|'.join((at_rate, url_link))

def preprosessing(text):
    soup = BeautifulSoup(text, 'lxml')
    new_text = soup.get_text()
    strip_text = re.sub(combine, '', new_text)
    
    
    # remove hashtag
    strip_text = re.sub(r'(\s)#\w+', r'', strip_text)
    
    try:
        clean = ''.join([i if ord(i) < 128 else ' ' for i in strip_text])
    except:
        clean = strip_text
    lower_case = clean.lower()
    
    lower_case = re.sub(r'[()\"#/;:<>{}`+=~|!?,]', '', lower_case)
    
    # (/[&\/\\#,+()$~%.'":*?<>{}]/g,
    
    # During the letters_only process two lines above, it has created unnecessay white spaces,
    # I will tokenize and join together to remove unneccessary white spaces
    words = lower_case.split()
    return (" ".join(words)).strip()

## Process all tweets

In [155]:
df['processed_tweet'] = df['tweet'].apply(preprosessing)

In [176]:
df['is_Threat_user'] = 0
df['subject'] = None

#### 1.Email

In [177]:
count = 0
for row_no,tweet in enumerate(df['processed_tweet']):
    if re.findall("[a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+",tweet):
        print(row_no,tweet)
        df.loc[row_no, 'is_Threat_user'] = 1
        df.loc[row_no, 'subject'] = "email"
        count += 1
print(count)  

4054 i want a new laptop. hp tx2000 is the bomb. who knows how much it is im me gabbehhramos@yahoo.com
7917 who stole elledell@gmail.com
8496 really did you send out all the info already if you did..maybe you could just email me stuff missataari@gmail.com
10290 awh...that's kinda sad lol add me hello.kitty.65@hotmail.com
16413 got 2 bottom of it human error bug from a release last month being fixed tonight. email press@linkedin.com for details
61735 sumthin u noe how sum girls are lyk an hourglass or pear...im a coconut tree not cool dude lol -&83w@r3...-&
65407 rb ...run out of ya allocated props need to rb for@dickadcock@bendrix@corts@starcrazy @2fast4u@s... 5igvc
72386 madonnafae ok i found this on a fluff page email nrqj8ga@facebook.com if your acc has been disabled.your completly gone from fb x
74295 i can't dm you my email cos you're not following me but its danielleloren@live.co.uk anyway.
87268 but...@floppyflopsome...can't you come for just one weekend ps. i am so confused too

620564 i tried everything. i emailed support@twitter.com no answer yet
629682 hmmmmm daniel is eating. sounds funnnnnnn uugghh hmm my tummy kills. danil e mail me at gabby.2233@hotmail.com
637243 sara@pigmystrong.com i'm about to lose my hosting
639952 midsummer but only enjoying@work...
654445 send it to bath.csc@firstgroup.com and my colleagues there will pass your comments on to the area ops director.
658094 ahhhh no one has approved my app yet hollywoodbound@gmail.com i already emailed eric but i wanna start now
663083 bow got a point this shitttttttt&myspacefacebook this shit boringletoy@luckett.goliveon ustream soonagin givea niggia sumthing2 look @
664044 hey q whats up look i gatta tolk to ya my email is tvworldrecords@hotmail.com so i realy wont u to add me am so tired
673941 went back and checked email. had a recipient failed permanently message. email me brianadrian@gmail.com
676251 i'm so tired who wants to talk hellossoolciz-@live.com.ar
676507 sup yall i need sum friends.

882669 add msn baru please yg lama udh over limitednathaniaaa-@hotmail.com thnkxxxxxxxxxx
885294 if you'll send an email to support@footnote.com you can be our test case for sorting that out
891583 feel free to email the request directly to me - chris@js-kit.com for help with js-kit
891847 cant be bothered with twitter anymore today aha talk to me on myspace or msn instead byeee twitterland x - stacey1889@hotmail.co.uk
892658 i can help you further if you like add me to msn m1k30rz@gmail.com
893177 sure that would be fine i'm pretty sure my email is - milaaayxo@aim.com not sure. i dont check it a lot.
896163 i'm going to bed. still tired from last night's 5 hour broadcast text me nice stuff cell@obnoxiousacorns.com yes it goes to my cell.
896177 good night for now. keep sending your requests to truecarvalue@gmail.com i'll post your value range asap i promise.
899787 i love gifts *giggle* send to princesspw@gmail.com
902281 not swine flu i hope take care of your health mate tell me abou

1037936 send that lightning video to me.. i'll post it for all to see amywoodtv@gmail.com
1038722 agentmp_04@hotmail.com.. thanks alot p.s randoms send me spam wooo
1038969 wgworldwide@gmail.com
1042442 fire an email to editor@fuelyourapps.com w a bio and some proposed topics and we will discuss.
1046000 okay someone must have a google wave invite to send me dominichodgson@gmail.com
1046846 hey you are nice check my profile. i can send you my pics tell me your email. or write me gmail_all@mail.ru
1049831 friday follow @heathenshearth@littlereddoor@bohemeanne@islaluna@littlepapoose@kreativlink@rachelstyle... hoping that worked.
1050315 follow me...@tashi316...promise to make you feel good...pinky swear
1051247 yes u can contact dennis mitchell at coach991@gmail.com. that would be awesome
1051364 i'm interesting in the english training at uk. my contact email address is orapan.sphere@gmail.com thks fr yr kind support.
1054997 peace you can send through music at anytime for show considera

1197525 my dad fails at life oh clearly i will see and yeah its dominic_mc@hotmail.co.uk
1201708 i'm bored hit me up some1 send me sum i wanna see kprincessw@tmail.com
1202699 we could actually conduct it via gmail alexander.ebony@gmail.com . what exactly did you want to know just curious
1203654 hi-it's cool. ya i have msn- apple_chic@live.com
1204771 who got a sidekick hit me up amaniy@tmail.com pow s.o.d. && p.t.e. thats what i repesent until the end...
1207525 i think it's time for this tweeter to go to sleep email me tits or tats who.is.tribute@gmail.com and get followed
1207741 would definitely love to email me it stacey.ess@gmail.com
1209472 added you. mine is mogz@me.com
1209552 what do you think about doing a video like this and send to endushunger@gmail.com
1212638 you just need to register. then email your blog to victoria.thompson@emap.com - look forward to it
1212761 new friday weekly in the east bay starting july djs & production companies interested hit up info@omnieraen

1308990 hello i'm tricina & i don't bite cinabun@att.blackberry.net pin24a4e3db contact mee. i like a good laugh.
1310330 just got an email from abuse@bluehost.com - must trim my 300k files to 50k. that's fine but how did they let me get that far fun.
1311304 airra_vea@yahoo.com add me up fs account en ym
1312719 night tweets bless my inbox istarra@gmail.com singers producers...other writers....lol we just hit me up w your offers
1312771 night tweets bless my inbox istarrah@gmail.com singers producers...other writers....lol we just hit me up w your offers
1313197 anybody need some copywriting looking for some freelance work. my mail lovenadav@gmail.com
1319480 okies..kimkin90@yahoo.com add k
1321017 please add my friendster account angelique15borbon@yahoo.com tnx guys
1321121 if you love mcfly then add me on msn and we can chatt serenamcflyfan@hotmail.co.uk xx
1322130 send your most interesting family holiday pics to pics@gotravel24.com. just let us know where it was taken and who is i

1482020 if you have any video of us email them to joemwestbrook@yahoo.com
1484866 my gamer artistic 6'2 19 yr old nephewgodson needs distraction. any hot chicks wanna send him a saucy pics. sasquatchalex@cogeco.ca
1488996 add nois no msn povo twitabbo@hotmail.com
1489424 cool grissylove@gmail.com
1491389 live chat in 7 minutes you have only 30 minutes for talking with me.... come onnnnn hahaha email jordanpruitt0@yahoo.com
1491561 i have email i will reply contactjh24@gmail.com
1492361 yeahh but im being slow to reply atm joshypearson@hotmail.com
1495696 or the ones sounding like name01@ispname.net.
1497751 awesome we need to work together sometime i work with all the time. info@bryankannowski.com
1500011 that page is a fake...i was told this was his real page...@y0itzbreezy....reliable source told me that one
1506637 lol sorry its m_randazz@yahoo.com bb
1507468 hey i thin i didn t leave my e mail adress right oh well it s hector14517@hotmail.com hope u can add me. byeee
1507719 sellin

#### 2.Age / Birth date

In [178]:
count = 0
for row_no,tweet in enumerate(df['processed_tweet']):
    if (re.findall("(?:date of birth|my dob |b - day|my age |years old)",tweet)):
        print(row_no,tweet)
        df.loc[row_no, 'is_Threat_user'] = 1
        df.loc[row_no, 'subject'] = "Birth"
        count += 1
print(count) 

1497 i'm only 540 years old in pixie years.
8205 just got a phone call from my mom. mine and great grandmother passed away today she was 95 years old. rip abuela fela.
15522 my phone is over 3 years old it can't do crap and i gotta pay for texting i can't hold a call for more than 10 sec con't.
34609 he's fuckin 22 dis year shit fuckin asshole shit years older ppl who r fuckin fit hve ta b older dan me.
39786 is 21 years old and about to end her time in florence
41629 that's what happened with mine too. she was 18 years old...but had to be put down. i cried for two weeks.
42355 i've got the blue screen of death frequently on my poor comp. it's 4 years old & needs to be put to sleep. i feel your pain
44829 i hurt my foot when i went jogging with 4 years old running shoes. yippee. i can't put any weight on it.
47547 seriously upset by a lady's comments last night - she said her mates were wondering about my age i was either 28 or 38... i'm only 27
51585 oh yum also. possibly i squeed a l

325496 sad cuz my dog is dying ive had it since i was 5 years old which makes her about 125 in dog years... she lived a good life
327632 lol romy he was like 2 years old but his mom kept pulling him by his arm all rough...ugh
330342 i'm 21 years old on my birth certificate... but only 12 years old in my pants...
331617 my nieces are almost 4 and 2 years old..which makes that extra creepy
336335 my grandfather died yesterday. he was 96. in the room next to me a 16 years old boy who have a cancel... life is unfear
337512 i'm just lucky that most of the people i work with are my age and very openminded. *hugs*
344150 i think i'm gonna die if i see another smurf. also if i don't speak with someone my age help.
344406 watching the nhl now a days is like watching the wwf when we were around 10-11 years old. via yeah sucks
346741 i am huge fan of you and i think we have the same character...greetings from germanyi'm 16 years old...want to be like you
362038 so do i. but trying not to get my h

581570 i'm making a get well soon card for a little boy with the swine flu. it's so sad he's only 8 years old i think
586539 find another person just as perfect who is my age lol
588352 got mistaken for a high schooler.. again. i need a makeover hey i know tlc should do a new show ten years older.
590981 my oldest is going to be 5 years old in less than a week they grow up so fast
596621 rip aunt marge. she was 103 years old. and the true meaning of life.
597956 im plotting to kill my brother hes 27 years old & he still lives with me and dad & yeals at me 4 whispering on the phone
599562 itv passed... but i have to fix 2 lights. but now my car is 10 years old and i have to pass the test every single year
605333 he will be 9 years old in september sure kat is growing fast too.
616992 no i was like 8 years old when i had it
617249 goddammit. i haven't lived in northern me since i was 3.5 years old...that doesn't help me now
620776 i think you should get me a new tv mine is like 50 years 

837432 haha..kay...she's 10 years old and i will do some damage hah
839269 that would be my son without a doubt he's 12 years old and the air that i breathe & the love of my life
846916 just gave a 30 min. distance learning course on how to use flickr to the 70 years old mother-in-law it worked
847777 i bought it i bought it i haven't had a bike of my own since i was 10 years old
850117 and i do it daily ...particularly today. i know this is hard to believe but i am 72 years old today. ha
855027 its my birtday today and i am now 45 years old. where in the hell did all the time go does anyone know cause i sure dont. sml & gb
855028 going to skype i see i still have my old pic there - 3 years old atleast
859486 o elmo loves you lol follow me i'm a katy perry fan 11 years old. yup
866050 watched my daughter dance in a ballet recital and she did an excellent job. best in class - at five years old.
866331 got a comment from 57 years old podcast listener. 'dance music for adults' for real.
8

1108024 im just come back from my mum friend's house i met a very cute little boy. he's just a 3 years old i guess
1112471 @1shawnl luv ya still girlie. i mean u iz 30 years old & shit. i expect u too cook breakfast. smile hun.
1114976 im a pc and im 8 and a half years old
1118724 al green won i've had a crush on him since i was like 3 years old
1119317 hahaa he's a pc and he's 5 years old.. well news for ya im a pc and im 12 years old
1138116 i have two 2 years old and it's just funny 2 see them danceit doesn't matter what the music is lol luvin the baby dancin'
1139801 have always loved dragons... is that silly at 37 years old
1162308 cant wait it's gonna be even better than last time i'm all old tho wait til you get to my age lol
1163796 hey hunni..i was just wondering hw old u are..see if ur similar to my age xxxx hope ur well xxxxx
1164240 ohhh im 19. i dont care who knows my age lol
1166211 - i am truly 8 years old
1166212 big ben - 150th years old today
1172379 yeah i remember m

1389657 tetris is now 25 years old and still going strong.
1392228 oh your english is quit good and i don t know if sims 3 run on my computer because it s already 3 years old...
1393389 i love my age where i can talk to my parents wo drama...1 pt for adulthood
1396204 me too that and what's my age again have got to be my favourites
1396329 yes they are my boys 8 and 7 years old
1398645 just played tetris.. it's 25 years old today and i remember playing it when i was very little. i still like it
1408917 . tuesday is my day 23 years old
1414060 and i loved to know american teenagers of my age and you seem so cool
1414749 hehe he forgot how old i am. i love that he is 9 years older.
1415663 yeppers i started tapping when i was about 5 years old
1416530 11 years old boys bragging about winking at mrs. bruni. they're young but not that young.
1419083 the bad news our clothes washer died it's 17 years old. the good news we're buying a new clothes washer today to help the economy
1420574 that

#### 3. Bank Account no 

In [179]:
count = 0
for row_no,tweet in enumerate(df['processed_tweet']):
    if (re.findall("(?:my debit card|my credit card|my account no|my account number|my bank)",tweet)):
        print(row_no,tweet)
        df.loc[row_no, 'is_Threat_user'] = 1
        df.loc[row_no, 'subject'] = "Bank"
        count += 1
print(count) 

7644 first time to go negative in my bank account i am not a happy camper what was i thinking
7843 just had my credit card pwned luckily i won't have to cough up for it but still a pain going through the disputation process
8513 lost my bank card... again. that's twice i've replaced in the last 2mos. i nevvver do this don't knw what the deal is grrrrr
10660 good morning what a nice day in the a. i'm on the phone paying bills right now & emptying out my bank account
15178 my credit card
28484 sadly i ended up emptying my bank account for photoshop
39484 horrible night of poker last night. burned off 13 of my bankroll. back to the grinding
40690 i havent dared check my bankaccount yet. its gonna be horrid i know it. i cant believe you cant make it...noooo
43767 but who will be my banker
45141 not much of a lie in but its all i can manage. would like to do something with my bank holiday weekend but think its unlikely that i will
47116 emusic renewal day is here but i lost my bank card 3 w

297949 i feel ya. my wish list is much greater than my bank account.
299188 doing my bank rec. hate doing my bank stuff with a passion i can't tell you
301404 omggggggg but u have to wait on ebay right just buy from retail damn my credit card i maxed it
306535 i lost my credit card
316818 man.. i went hard this weekend. checkin the scale is like checkin my bank account. i don't wanna see this weekends damage
319282 god i am so stupid sometimes only sometimes i thought sum1 used my credit card when it was stolen but turns out it was a direct debit
321998 i wish i hadn't forgotten my bank card. i have about 2.50 in my purse and i want sushi
324184 i swear my bank has had the same cd on replay for a year. currently on hold
328291 i maxed out my credit card in jack wills
336787 i hope $100 tickets help stimulate the economy because it sure doesn't help to stimulate my bank account
336888 so my bank account is writing red numbers again.that's what school fees for canada already do to me.cou

625098 more sleep wow account runs out tomorrow. my bank account is empty while moms care cup is empty.
627521 - i wish i was in a band. and i wish i could find my debit card.
632113 i'm looking for a new small camera backpack. for the good of my back certainly not for the good of my bank balance.
633707 no food for me i'm soooo hungry anyone hungry i reallly need to not use my credit card so much
636786 someones nicked twenty quid half of my ttkp fund out of my bank. unless i paid my mum back the whole thing oh god im so stupid
637404 yeah but then i don't get the cheap pick up price. plus i only have my debit card on me
640802 looking at my bank account..have i really spent that much money since i've been home
641325 having to block my debit card because i'm sure there's been fraudulent activity on it. bugger
643749 me but i dont know my account number
645149 i know right as i just said.. now or never but who pays for it i gotta go talk to my bank...
645444 my bank just told me that 

1314083 oh dang my bank card is bout ta look active
1317164 nicepaul i could upload it to youtube under my account no one ever watches my stuff
1317809 off to play with my credit card
1318861 so long not my bank
1322694 ok basically just emptied my bank account at the vip party will make a video shortly on it
1355818 ok mom lets use my credit card and try to tell me you didn't and then i can check my purchases and see that you did bitch
1357512 i was short 35 cents so the panda girl gave me a buck off so i didnt have to use my debit card sweet there are good people left
1359648 i am disappointed as usual. and my bank account will be as well after i use this shopping spree to make me feel better
1360934 i but mailing me my bank balance on friday evenings is just not cool. how about let's say monday mornings
1363450 ok i need to go do some damage to my debit card tweet ya soon
1364024 i love my bank and my bank loves me
1386551 lol - just noticed that you asked me that question at my acc

#### 4.phone number

In [180]:
count = 0
for row_no,tweet in enumerate(df['processed_tweet']):
    if (re.findall("[a-z. ]+ \+?1?\s*\(?-*\.*(\d{3})\)?\.*-*\s*(\d{3})\.*-*\s*(\d{4})$",tweet)):
        print(row_no,tweet)
        df.loc[row_no, 'is_Threat_user'] = 1
        df.loc[row_no, 'subject'] = "phone"
        count += 1
print(count) 

29265 boyfriend came over. but we have yet to say one nice thing to eachother help text me if u have suggestions 7142258138
29768 ok im getting upset call me lollll 4434156058
34625 at schoool text and cheer me up 412-992-6991
54722 la la la 3 i need a txt. i'm lonely 540.903.2373
71132 text im bored 419-961-0631
72848 what about texting i'm really bored right now. 715-923-1678
78900 why did everyone i was texting me stop well text me...ladies. 626 374 5195
127374 worst movie ever marley&me...... .......... i cant handle it ....i luv u rosie 3333333333
132067 does that mean that there's 6707 of me if there's one of me in every million i thought i was 1 in 6706993152
181633 im so angry im gunna cry again someone txt me 1714-457-7893
183805 going to the dog races tomorrow.. need companyy. anyone up to it should text me 8134945146
206564 - the family is here helping me move now. hostility is in the air... text me 254-247-5319
208415 just got my cell working kind of. i lost all my numbers 

1145144 going to alexandre's in oak lawn with alex. join us or text me 214.436.3348
1192784 thank you during the show you can always call in to ask a question if you want as well toll free 877.953.3990
1193015 new extensions text me 0433160532
1202618 waitin for mtv movie awards to come on....this shall be good. text mehh. 9852460068
1204327 twilight crew looks amazing at mtv awards so very proud of them 3333333333
1206306 will be in the sga office on mwf from 2-8 & on tues & thursdays from 5-9. come say hi calltext him anytime 3018029845
1208251 goin to catch my z'sssss... hit me up 404-667-9431
1212349 awww 1444444443
1213260 call me 0448417513
1222309 good morning guys back on the air gonna do the morning 3 some in a few call and win some tixx 1888 414 2104
1224185 stop cell phone telemarketing calls by registering your number at the do not call registry takes about a minute 888-382-1222
1226441 mark lee - another good show todaythx for all ya feedback via txt & chatroom ginge after

#### 5.Address

In [181]:
count = 0
for row_no,tweet in enumerate(df['processed_tweet']):
    if (re.findall("(?:my location|my current location|my home address|my house address|i live in)",tweet)):
        print(row_no,tweet)
        df.loc[row_no, 'is_Threat_user'] = 1
        df.loc[row_no, 'subject'] = "Address"
        count += 1
print(count) 

74 why won't you show my location
1504 i'm not excited as i live in wales
2642 ok here's the deal i live in a small town i've never seen anythin famous n anyway so i thought i might luck up on something
3123 - i love mountains.. but i live in belgium..
4720 i hope you realize that your website is blocked by china.. i live in wuhan and have to proxy in to your site
4944 i really wish i could see ya'll tonight but i live in lansing so i had no way of entering in the contest sigh
8425 that dancing with the stars sound fun but i live in the upper michigan and i wish i could go.
11452 i would lol but... i live in london
14138 ugh why don't i live in california coachella's lineup is sick. fml
16866 i live in the country... nearest neighbor is like 100 feet away
18045 hey tom hope you guys have a great tour i live in australia so i cant go
18389 yeah ur lucky bevrely hills i live in new york but havnt run across anyone famous yet
19796 didn't go to coachella because i live in the middle of no

116960 ok i will add you but where do you live cause i live in northern ireland
117073 omg. that so far away o. i live in london
118537 i would if i could but i live in ri not mn
118997 why can i find the small town i live in in the weather widget on osx but not in the weather.app on my ipod
118998 why can i find the small town i live in in the weather widget on osx but not in the weather.app on my ipod
120447 i don't have a yard i live in an apartment style condo
120884 lucky i wish i could have felt the earthquake...but i live in oklahoma
121316 - me too but i live in hungary
121408 i wanna be in an earthquake i live in virginia though
122457 if you stay in la long enough you may feel earthquake. i live in sf bay area & felt earthquake many times. but it's no fun
122749 i've been a fan of urs for 3years i no pretty everything bout ur music n acting stuf i never bought any cuz i live in morocco
124456 oh really. i live in australia so i didnt hear about it .. sad to hear hope everythi

220824 awww booo i live in van nuys but won't be home tonight we needa hang soon joesars lives up the street from me
228430 yeah i will miley if only i live in the united states
233861 cant wait to see my fam in a couple of weeks. never been away from my mum for more than 2 weeks before.. now i live in another state misu
234903 johnny cupcakes speaking at leeds why don't i live in leeds again
235237 and am well gutted coz yuuh doin a concert on july 2nd ma birthday n i can't get there mostly cause i live in england x
235585 @_j_a_m_e_s_ i loooove subway its soooo goood -p but i live in a huckville i only get when i am in a city
235729 says i want my own dog...but i live in an apartment not a good environment for a dog right
239567 nicee i've never been there i live in the north west part it's like scotland everything is green lol
246994 i wish i could live in la i wanna watch the mtv movie awards but i can't coz i live in fuckin boringg hungary it's not faair
250731 i wish i live in us

363922 i wish i was i live in florida though and there was no dc games this year bumer i 3 yall though
365593 wish i could b there2 bad i live in d otherside of d world
366037 hey man i wish i could go but i live in canada so i cant let me know how the concert is and if miley cyrus sings or not.
367202 me neither and i live in la. i wanted dallas. oh well...
367766 totally wish i was there but i live in nc and its raining outside
367951 it would be so cool to enter to be in your music video but i live in ohio and there is no way i can afford a plane ticket
368708 free donuts & ice cream why can't i live in the us and you're going to lolla i can't... *cries*
369156 it would make my year if u replied saying hi or somethin like that i live in the uk so havnt seen u live plz reply x
369404 i'm sorry i wish i could drive you but uhhm i live in iowa
369617 protip firefox 3 eats about 500mb less ram after clearing 3 months of history. my location bar is much less useful now though
372838 i wo

486293 wow i'd love to go there but i live in glasgow so its a tad far away
486331 wish i could i live in london bit too far away x
487570 wish i could go but sadly i live in shity scotland are you touring in the uk next year
488628 id luv 2 but i live in ireland nearly 2million followers xd
490460 course we can be best friends haha tweet buddies but i live in birmingham
490849 went onto the teen choice awards to vote for you and cos i live in the uk it wouldn't let me sorry
490850 went onto the teen choice awards to vote for you and cos i live in the uk it wouldn't let me sorry
492499 aw god i live in chicago but i was in school lol
494007 i would love to win but i live in other country
494778 how many hours until mantaince i live in a different time zone. somebody help me
495931 i want to go but i live in florida
498691 however at home ill still use twitter lol i live in a deadzone soooo sad
499607 i want to but i live in ct are you going to boston or nyc
500344 if only i lived anywh

...

707337 and still i'm gonna be homeless after 2 months cant i live in the attic
707407 wow tila nice flyer sry but i live in german
707722 . i hope you get better. i live in ct but i didn't get tickets to see your show. im so upset. but i know you'll do great
708466 unfortunately not. i live in italy and i couldn't go did you go
708555 omg his mum lives selsdon byron road u kno it i like croydon too good vibe good shops and no cus i live in essex now xx
708640 i wish but i live in mexico
708882 film three of harry potter marathon. i have to leave as i live in bloody surrey. little whinging. well a fair amount of whinging.
709640 set my location to carlton vic in bing and yet it still showed me sydney results disappointing
711051 hot like a sauna no lie windows open doors open... nothing. i actually thing it got hotter i live in an attic
711172 i want demi to tweet me since im never gonna meet her cz i live in ireland
711470 do u like my profile picture i think it's adorable 3 i want tha

832790 i live in sweden but i m really good in english but tomorrow i got like the biggest test in english speaking i really hope i pass it
838593 im jelous im just going to one x i live in sweden i never thought they would come here but they are so excited
839735 well since i am not in switzerland i have no idea i live in central pennsylvania in the states
840501 wow mileyy i live in germany .. you were so neaar and know soo far .. do pics from you in twipic wanna see you in germany
840938 hii just thought i'd tell you that you're super amazing your videos r so funny lol an i live in ohio too
841490 i live in san diego but that was in vegas
844944 hope that you have a great time in italy i'm italian but i live in germany xoxo s
853242 speeding tickets just knowledgeable driver right another reason i live in pdx and generally don't drive.
854520 actually no though that's what i plan to. i wanted to ask you how's life for a student in monterrey. i live in df by the way
856828 i'm going 

1056156 nitrojohn how can you eat there i have been there twice and got sick both times i live in laverne id rather got to zendajas
1058108 i live in south central pa. york
1059148 haha my light is definitely radioactive. i live in the neighborhood of a nuclear reactor btw are you able to see in the dark
1065802 i just wanna thank u for all that u do 4 us. i live in argentina and u definitely make me feel distance is nothing luv u
1068513 haha nope i live in the usa so i havent been to any yet im hopeing someday soon theyll come here
1072432 argh i live in australia so i never got to listen to your new songs. but i can already tell they're awesome.
1074512 i live in washington dc with my love n my brother n he's dating my best friend i could not ask for anything more
1081360 uhhh i just saw that you asked if i was going to tomorrow's show and unfortunately no as i live in mn not tx
1081791 . does that mean i can't like the texas longhorns since i live in colorado cause i do.
1084589 i 

1309705 i live in victoria bc so we have the heat too - love it
1311776 i'm into yoga but we have no yoga teachers where i live in kosovo...using dvds instead which is better than nothing
1313633 heyy. tyra and ellen i love your shows id like to come on one of yalls shows thankss by the way i live in texas
1318566 i don't blame u either... 'tis certainly hot out there lol. yes i live in exmouth & work in exeter @ university
1319026 hi megan i live in aus.. i think you're brilliant i can't wait for transformers to come out in theaters here ciao
1322412 just spent my evening being a tourist fun wow sydney is truly stunning i feel very special tonight knowing i live in a beautiful city
1325544 who.. lily or pink pink at sydney entertainment centre lily at horedern pavillion ... i live in sydney
1326603 i live in the most isolated city in the world haha if u need a true escape.. ppl here so don't give a shit who ppl are
1327318 i live in malaysia.malaysia good.hehehot and humid as always. 

1540885 i live in a tv box apparently nd my friend jacqui lives in a poptart box i throw it on the floor though nd go haha jacqui ]
1543205 i live in my head. because reality is so overrated.
1543765 i live in southern california. in los angeles county.
1544789 i live in dubai this is so awesome you're actually coming lots of love
1552043 no sound from outside may be something to do with the fact that i live in the country nearly as funny as jimmy carr's laugh not quite
1554742 hi i live in brazil i win i amo icarly to jannette and nathan they are very nice too. i hope you in brazil beijos
1557196 - awh yall are both precious i love you miley i live in ga maybe i'll see you before you go
1557348 done and done. i take no chances with my electronic toys i live in the lightning capital of the world
1558278 you're coming to australia woohoo work or holiday i live in sa and would love to see you
1559938 i wish i live in nyc another 2 years baby and i will be headed that way
1560221 yepp i l

In [185]:
df[df["is_Threat_user"]==1].shape

(2426, 5)

# Modeling

# Evaluation